original Polars TPC-H code: https://github.com/pola-rs/tpch

In [1]:
import uuid
import polars as pl

from ibis_bench.utils.monitor import monitor_it
from ibis_bench.utils.read_data import get_polars_tables

from ibis_bench.queries.polars import *  # noqa

In [2]:
# YOLO
import warnings

warnings.filterwarnings("ignore")

In [3]:
sf = 1
n_partitions = 1
lazy = True
streaming = False

customer, lineitem, nation, orders, part, partsupp, region, supplier = (
    get_polars_tables(sf=sf, n_partitions=n_partitions, lazy=lazy)
)

In [4]:
monitor_it(
    q1,
    sf=sf,
    n_partitions=n_partitions,
    query_number=1,
    system="polars" + "-lazy"
    if lazy
    else "-eager" + "-streaming"
    if streaming
    else "-batch",
    session_id=str(uuid.uuid4()),
    lineitem=lineitem,
)

INFO:root:running and monitoring for system polars-lazy query 1 at scale factor 1 and 1 partitions (session id f5da855d-cebe-444e-8eb7-0c18ed46f47a)...
INFO:root:	writing polars (lazy) result to results_data/polars-lazy/q_1.parquet...
INFO:root:	writing monitor data to gs://ibis-benchy/benchy_logs_v0/bf82dfec-35e8-46a9-add1-0d1b838e6306.json...
INFO:root:	done writing monitor data to gs://ibis-benchy/benchy_logs_v0/bf82dfec-35e8-46a9-add1-0d1b838e6306.json...


In [5]:
monitor_it(
    q2,
    sf=sf,
    n_partitions=n_partitions,
    query_number=2,
    system="polars" + "-lazy"
    if lazy
    else "-eager" + "-streaming"
    if streaming
    else "-batch",
    session_id=str(uuid.uuid4()),
    nation=nation,
    part=part,
    partsupp=partsupp,
    region=region,
    supplier=supplier,
)

INFO:root:running and monitoring for system polars-lazy query 2 at scale factor 1 and 1 partitions (session id 97827773-a444-4e5d-abec-6c46eef51f89)...
INFO:root:	writing polars (lazy) result to results_data/polars-lazy/q_1.parquet...
INFO:root:	writing monitor data to gs://ibis-benchy/benchy_logs_v0/bf82dfec-35e8-46a9-add1-0d1b838e6306.json...
INFO:root:	done writing monitor data to gs://ibis-benchy/benchy_logs_v0/bf82dfec-35e8-46a9-add1-0d1b838e6306.json...
